# Arewa Data Science Deep Learning with PyTorch Capstone Project - Group 2

## WildLife Classify App


This notebook serves as the documentation for the capstone project (Group 2) for the Arewa Data Science Deep Learning with PyTorch Part 1 [course](https://github.com/arewadataScience/ArewaDS-Deep-Learning), instructed by [Mr. Mustapha Abdullahi](https://github.com/mustious). We applied our learning in Computer Vision to develop a model capable of identifying wild animals, utilizing the African Wildlife dataset for this purpose.We also deployed the model on streamlit.

We begin by the necessary imports. We use wandb to log all the training metrics.

> Densenet was selected due to its smaller size and efficiency compared to Alexnet. We could host the trained model weights on github for deployment on streamlit share. 

> Notebook was used in Kaggle on a Kaggle Dataset. All paths need to be properly redefined if one wants to use it in a different enviroment. Same result will be obtained nonetheless. Find it [here](https://www.kaggle.com/code/lukmanaliyuj/african-wild-life-classification) on kaggle

# Imports

In [2]:
# Necessary imports
try:
  import wandb
except:
  !pip install -q wandb
  import wandb
# importing split-folders to split my dataset
try:
    import splitfolders
except:
    !pip install split-folders[full]
    import splitfolders  
# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms,datasets
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from pathlib import Path
import os


# Logging into wandb (Weights and Biases)

Next, logging into wandb. This enables us save the model metrics in an easy to retrieve way.

In [3]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Setting up device agnostic code. Since we intend to use GPU to train the model.

# Setting up device agnostic code

In [4]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

We check the type of GPU device available on this kaggle notebook just to be sure.

In [5]:
!nvidia-smi

Sat Feb 17 12:35:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
NUM_WORKERS = os.cpu_count()


# Creating Utility Functions for Training

Next, is a couple of utility functions to make the training easy. These are functions we were given in the course [material](https://www.learnpytorch.io/). The first is a function for creating dataloaders from image directories (train and test directories).

In [7]:
def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

The next functions define the train and test steps and a function that combines the entire training in a single function, utilizing all the initial functions. 

# Training Function

In [8]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval() 

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]} 
    For example if training for epochs=2: 
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )
        # Log metrics to wandb
        wandb.log({"epoch": epoch+1,
               "train_loss": train_loss,
               "train_acc": train_acc,
               "test_loss": test_loss,
               "test_acc": test_acc})

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

We then define the folders for the input and output, using the directory structure in kaggle.

# Defining the Directories and Splitting the Images into train-test splits

In [9]:
input_folder = "/kaggle/input/african-wildlife"
output = "/kaggle/working/"

The splitfolder library is then used to split the images (already in an ImageFolder structure) into train and test splits. 

In [10]:
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(input_folder, output=output,
    seed=1337, ratio=(.8, .2), group_prefix=None, move=False) # default values


Copying files: 3008 files [00:29, 102.30 files/s]


The train and test directories are then defined from the output from splitfolder. 

In [11]:
# Set up train and test dirs
train_dir = "/kaggle/working/train"
test_dir = "/kaggle/working/val"

# Defining Data Transform

Next thing is to define the data transform we want to do on the images. 

In [12]:
# # Create simple transform
data_transform = transforms.Compose([ 
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    
])
# Create training and testing DataLoader's as well as get a list of class names
train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform= data_transform,
                                                                               batch_size=32) # set mini-batch size to 32

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7cf938e51ed0>,
 ['buffalo', 'elephant', 'rhino', 'zebra'])

The classes show up nicely. As a reminder, the intention is to build a classifier that classifies any picture into buffalo, elephant, rhino and zebra. 

Not knowing how the text files in the image folder will impact training, We remove them in the cell below.

In [14]:
# removing all the .txt file
datapath = '/kaggle/working/'

# Walk through the directory
for root, dirs, files in os.walk(datapath):
    for file in files:
        # Check if the file is a .txt file
        if file.endswith('.txt'):
            # Construct the full file path
            path_to_file = os.path.join(root, file)
            # Remove the file
            os.remove(path_to_file)
            

# Get The Pretrained model


Utilizing pretrained models is a popular strategy not only in Computer vision but in all of deep learning. We follow suit by utilizing the densenet201 from torchvision using the code below. 

In [15]:
# Setup the model with pretrained weights and send it to the target device
weights = torchvision.models.DenseNet201_Weights.DEFAULT # best available weight
densenet = torchvision.models.densenet201(weights=weights).to(device)


Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:06<00:00, 13.0MB/s]


We then look at the nature of the model using the summary function from the torchinfo library. 

In [16]:
summary(densenet,input_size = [32,3,224,224],col_names=["input_size","output_size","trainable"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Trainable
DenseNet                                 [32, 3, 224, 224]         [32, 1000]                True
├─Sequential: 1-1                        [32, 3, 224, 224]         [32, 1920, 7, 7]          True
│    └─Conv2d: 2-1                       [32, 3, 224, 224]         [32, 64, 112, 112]        True
│    └─BatchNorm2d: 2-2                  [32, 64, 112, 112]        [32, 64, 112, 112]        True
│    └─ReLU: 2-3                         [32, 64, 112, 112]        [32, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                    [32, 64, 112, 112]        [32, 64, 56, 56]          --
│    └─_DenseBlock: 2-5                  [32, 64, 56, 56]          [32, 256, 56, 56]         True
│    │    └─_DenseLayer: 3-1             [32, 64, 56, 56]          [32, 32, 56, 56]          True
│    │    └─_DenseLayer: 3-2             [32, 64, 56, 56]          [32, 32, 56, 56]          True
│    │    └─_DenseL

Since we are not going to retrain the entire model, we freeze the feature layers in the pretrained model so that we only focus on training the classifier in the final layers of the model. The feature layers will be useful in identifying general image characteristics. 

In [17]:
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in densenet.features.parameters():
    param.requires_grad = False
# Check summary again after
summary(densenet, input_size= [32,3,224,224],col_names= ["input_size","output_size","trainable"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Trainable
DenseNet                                 [32, 3, 224, 224]         [32, 1000]                Partial
├─Sequential: 1-1                        [32, 3, 224, 224]         [32, 1920, 7, 7]          False
│    └─Conv2d: 2-1                       [32, 3, 224, 224]         [32, 64, 112, 112]        False
│    └─BatchNorm2d: 2-2                  [32, 64, 112, 112]        [32, 64, 112, 112]        False
│    └─ReLU: 2-3                         [32, 64, 112, 112]        [32, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                    [32, 64, 112, 112]        [32, 64, 56, 56]          --
│    └─_DenseBlock: 2-5                  [32, 64, 56, 56]          [32, 256, 56, 56]         False
│    │    └─_DenseLayer: 3-1             [32, 64, 56, 56]          [32, 32, 56, 56]          False
│    │    └─_DenseLayer: 3-2             [32, 64, 56, 56]          [32, 32, 56, 56]          False
│    │    

# Setting up the classifier for training

Finally we define the random seed and the classifier of our new model.

In [18]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)

# Recreate the classifier layer and seed it to the target device
densenet.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1920,
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True)).to(device)

# Loss Function and Optimizer

We also define the loss function also known as criterion and the optimizer. Adam is a popular optimizer and the go to loss function for classification (multiclass) is categorical cross entropy loss, hence the choices we  made. 

In [19]:
# Define loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(densenet.parameters(), lr=0.001)

We also define the number of epochs. This is an important hyperparameter that can be tweaked depending on the model performance.

In [31]:
epochs = 10

# Initiate wandb

We initiate wandb so that we can properly track the model performance. 

In [32]:


# Initialize wandb
wandb.init(project='arewads_capstone', config={
  "learning_rate": optimizer.param_groups[0]['lr'],
  "epochs": epochs,
  "batch_size": train_dataloader.batch_size
})


# Actual Training

The training is done in this few lines of code thanks to the earlier leg work that was done. 

In [33]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
densenet_results = train(model=densenet,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=epochs,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")
# Properly close the wandb session
wandb.finish()

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.4878 | train_acc: 0.8125 | test_loss: 0.8406 | test_acc: 0.6794
Epoch: 2 | train_loss: 0.4994 | train_acc: 0.8077 | test_loss: 0.8984 | test_acc: 0.6771
Epoch: 3 | train_loss: 0.4735 | train_acc: 0.8285 | test_loss: 0.8422 | test_acc: 0.6725
Epoch: 4 | train_loss: 0.4763 | train_acc: 0.8277 | test_loss: 0.8219 | test_acc: 0.6782
Epoch: 5 | train_loss: 0.5007 | train_acc: 0.8189 | test_loss: 0.8737 | test_acc: 0.6887
Epoch: 6 | train_loss: 0.5281 | train_acc: 0.7933 | test_loss: 0.8761 | test_acc: 0.6655
Epoch: 7 | train_loss: 0.4820 | train_acc: 0.8253 | test_loss: 0.9172 | test_acc: 0.6586
Epoch: 8 | train_loss: 0.5257 | train_acc: 0.8005 | test_loss: 0.8662 | test_acc: 0.6678
Epoch: 9 | train_loss: 0.4777 | train_acc: 0.8189 | test_loss: 0.9217 | test_acc: 0.6655
Epoch: 10 | train_loss: 0.4832 | train_acc: 0.8053 | test_loss: 0.9086 | test_acc: 0.6377
[INFO] Total training time: 90.516 seconds


epoch,▁▂▃▃▄▅▆▆▇█
test_acc,▇▆▆▇█▅▄▅▅▁
test_loss,▂▆▂▁▅▅█▄█▇
train_acc,▅▄██▆▁▇▂▆▃
train_loss,▃▄▁▁▄█▂█▂▂
epoch,10
test_acc,0.63773
test_loss,0.90856
train_acc,0.80529
train_loss,0.48315


# Saving the model


The next thing is to save the model since we intend to use it for the app. After all, the accuracy is fairly okay. The `save_model` function is defined below in the modular code spirit. 

In [34]:
# Saving the model


def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

Model is saved and we give it a befitting name. 

In [35]:
save_model(model=densenet,target_dir='models',model_name='densenetafri2.pth')

[INFO] Saving model to: models/densenetafri2.pth


# Moment of truth

And the moment of truth, seeing if the weights can be used on a fresh downloaded version of densenet201

In [36]:
# Setup the model with pretrained weights and send it to the target device
weights = torchvision.models.DenseNet201_Weights.DEFAULT # best available weight
#transform = torchvision.models.AlexNet_Weights.IMAGENET1K_V1.transforms 
densenet2 = torchvision.models.densenet201(weights=weights).to(device)

# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)

# Recreate the classifier layer and seed it to the target device
densenet2.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1920,
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True)).to(device)

model_path = '/kaggle/working/models/densenetafri2.pth'

# Load the saved model state dictionary into the model
densenet2.load_state_dict(torch.load(model_path))


<All keys matched successfully>

And, it's a success. The hardwork paid off. 

# Finally Prediction on New Data

In the final section of the notebook, we predict on a newly downloaded image from the internet.

First the paths to the model and the picture are defined.

In [37]:
image_path = '/kaggle/input/random-picture/pic.jpg'
model_path = '/kaggle/working/models/densenetafri.pth'

Then we define functions that allow us to load the model and predict on a new image.

In [38]:
def load_model(model_path, device):
    """
    Loads a model from the specified file path.

    Args:
    model_path (str): Path to the model file.
    device (str): The device to load the model on ('cuda' or 'cpu').

    Returns:
    torch.nn.Module: Loaded PyTorch model.
    """
    weights = torchvision.models.DenseNet201_Weights.DEFAULT # best available weight
    # Recreate the classifier layer and seed it to the target device
    model = torchvision.models.densenet201(weights=weights).to(device)
    model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1920,
                    out_features=4, # same number of output units as our number of classes
                    bias=True)).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

def preprocess_image(image_path):
    """
    Preprocesses the image for model prediction.

    Args:
    image_path (str): Path to the image file.

    Returns:
    torch.Tensor: Preprocessed image tensor.
    """
    image = torchvision.io.read_image(image_path).type(torch.float32)
    image = image / 255.0  # Normalize to [0, 1]
    transform = transforms.Resize(size=(64, 64))
    return transform(image)

def get_prediction(model, image, device):
    """
    Predicts the class for the given image using the specified model.

    Args:
    model (torch.nn.Module): The trained model for prediction.
    image (torch.Tensor): The preprocessed image tensor.
    device (str): The device to perform prediction on ('cuda' or 'cpu').

    Returns:
    str, float: Predicted class name and the probability.
    """
    class_names = ['buffalo', 'elephant', 'rhino', 'zebra']
    image = image.unsqueeze(0).to(device)  # Add batch dimension and move to device
    with torch.inference_mode():
        pred_logits = model(image)
        pred_prob = torch.softmax(pred_logits, dim=1)
        pred_label = torch.argmax(pred_prob, dim=1)
    return class_names[pred_label], pred_prob.max().item()

def predict_image(image_path, model_path):
    """
    Main function to handle model prediction on the given image.

    Args:
    image_path (str): Path to the target image.
    model_path (str): Path to the trained model file.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"[INFO] Predicting on image: {image_path}")

    # Load the model and preprocess the image
    model = load_model(model_path, device)
    image = preprocess_image(image_path)

    # Predict and print the result
    pred_class, pred_prob = get_prediction(model, image, device)
    print(f"[INFO] Predicted class: {pred_class}, Probability: {pred_prob:.3f}")

And lastly we predict on the image

In [39]:
predict_image(image_path,'/kaggle/working/models/densenetafri.pth')

[INFO] Predicting on image: /kaggle/input/random-picture/pic.jpg
[INFO] Predicted class: buffalo, Probability: 0.634


And it predicted the right class of the animal. Yay. Our small model works. 

# Deployed on Streamlit : Check it out

Seeing our small model works, we deployed it on streamlit. 
Kindly find it [here](https://wildlifeclassify.streamlit.app/) and give feedback on the performance. 

If you found this notebook helpful, kindly give it a thumbs up. 